In [ ]:
import numpy as np
import cv2 as cv
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import time
import torch
from pathlib import Path

from lac.perception.segmentation import UnetSegmentation
from lac.perception.depth import (
    stereo_depth_from_segmentation,
    project_pixel_to_rover,
    compute_rock_coords_rover_frame,
    compute_rock_radii,
)
from lac.control.controller import ArcPlanner
from lac.utils.visualization import overlay_mask, overlay_stereo_rock_depths
from lac.utils.plotting import (
    plot_points_rover_frame,
    plot_path_rover_frame,
    plot_rocks_rover_frame,
)
from lac.utils.frames import invert_transform_mat, apply_transform
from lac.util import load_data, get_positions_from_poses
import lac.params as params

%load_ext autoreload
%autoreload 2

## Obstacle detection


In [ ]:
segmentation = UnetSegmentation()

In [ ]:
data_path = Path("../../output/AutoAgent/arc_planner_test")
data_path = Path("/home/shared/data_raw/LAC/runs/double_loop_preset1")
data_path = Path("../../../output/RecoveryAgent/default_run")
initial_pose, lander_pose, poses, imu_data, cam_config, waypoints = load_data(data_path)

In [ ]:
prev_index = 490
prev_left_image = cv.imread(data_path / "FrontLeft" / f"{prev_index:06}.png", cv.IMREAD_GRAYSCALE)
prev_right_image = cv.imread(data_path / "FrontRight" / f"{prev_index:06}.png", cv.IMREAD_GRAYSCALE)
prev_left_masks, prev_left_labels = segmentation.segment_rocks(prev_left_image)
prev_right_masks, prev_right_labels = segmentation.segment_rocks(prev_right_image)
prev_left_seg_full_mask = np.clip(prev_left_labels, 0, 1).astype(np.uint8)
prev_right_seg_full_mask = np.clip(prev_right_labels, 0, 1).astype(np.uint8)

prev_results = stereo_depth_from_segmentation(
    prev_left_masks, prev_right_masks, params.STEREO_BASELINE, params.FL_X
)
left_overlay = overlay_mask(np.array(prev_left_image), prev_left_seg_full_mask)
left_overlay = overlay_stereo_rock_depths(left_overlay, prev_results)
fig = plt.figure(figsize=(10, 5), dpi=100)
plt.imshow(left_overlay)
plt.axis("off")

In [ ]:
index = 500
left_image = cv.imread(data_path / "FrontLeft" / f"{index:06}.png", cv.IMREAD_GRAYSCALE)
right_image = cv.imread(data_path / "FrontRight" / f"{index:06}.png", cv.IMREAD_GRAYSCALE)
left_masks, left_labels = segmentation.segment_rocks(left_image)
right_masks, right_labels = segmentation.segment_rocks(right_image)
left_seg_full_mask = np.clip(left_labels, 0, 1).astype(np.uint8)
right_seg_full_mask = np.clip(right_labels, 0, 1).astype(np.uint8)

results = stereo_depth_from_segmentation(
    left_masks, right_masks, params.STEREO_BASELINE, params.FL_X
)
left_overlay = overlay_mask(np.array(left_image), left_seg_full_mask)
left_overlay = overlay_stereo_rock_depths(left_overlay, results)
fig = plt.figure(figsize=(10, 5), dpi=100)
plt.imshow(left_overlay)
plt.axis("off")

In [ ]:
# TODO: use compute_rock_points and compute_rock_radii

In [ ]:
rock_index = 4
print(results[rock_index]["depth"])
x, _, w, _ = cv.boundingRect(results[rock_index]["left_mask"].astype(np.uint8))
width_x = w * results[rock_index]["depth"] / params.FL_X
print(f"Width: {width_x/2} m")

# big rock at 11000 time step: 0.55 m

In [ ]:
lander_local = apply_transform(invert_transform_mat(poses[index]), params.LANDER_GLOBAL)
lander_local

In [ ]:
prev_rock_data = []
prev_rock_points_rover_frame = []
prev_rock_data = compute_rock_radii(prev_results)
print(prev_rock_data)

for rock in prev_results:
    prev_rock_points_rover_frame.append(
        project_pixel_to_rover(rock["left_centroid"], rock["depth"], "FrontLeft", cam_config)
    )

prev_rock_points_rover_frame = np.array(prev_rock_points_rover_frame)
# print(rock_points_rover_frame)
fig = go.Figure()
fig.add_trace(go.Scatter(x=lander_local[:, 1], y=lander_local[:, 0], fill="toself", name="lander"))
print(prev_rock_points_rover_frame)
print(prev_rock_data)
fig = plot_rocks_rover_frame(prev_rock_points_rover_frame, prev_rock_data, fig=fig, color="red")
fig.show()

In [ ]:
rock_points_rover_frame = []
rock_data = compute_rock_radii(results)
print(rock_data)

for rock in results:
    rock_points_rover_frame.append(
        project_pixel_to_rover(rock["left_centroid"], rock["depth"], "FrontLeft", cam_config)
    )

rock_points_rover_frame = np.array(rock_points_rover_frame)
# print(rock_points_rover_frame)
fig = go.Figure()
fig.add_trace(go.Scatter(x=lander_local[:, 1], y=lander_local[:, 0], fill="toself", name="lander"))
fig = plot_rocks_rover_frame(rock_points_rover_frame, rock_data, fig=fig, color="red")
fig.show()

In [ ]:
print(rock_data)

In [ ]:
pose_difference = poses[index] - poses[prev_index]
pose_difference

## Arc path planning


In [ ]:
planner = ArcPlanner(21, 12.0, 0.6)


arcs = planner.np_candidate_arcs

In [ ]:
print(rock_points_rover_frame)
test_rock_points_rover_frame = np.array([[0.5, 0.5], [0.4, -0.4], [0.2, 0.3]])

In [ ]:
test_rock_data = [0.07, 0.06, 0.05]

In [ ]:
# print(f"candidate arcs shape: {planner.np_candidate_arcs.shape}")
waypoint = waypoints[index]  # would prefer a way to automate this


current_pose = poses[index]

test_rock_points_rover_frame = np.array([[]])
test_rock_data = []
control, best_arc, waypoint_local = planner.plan_arc(
    waypoint, current_pose, rock_points_rover_frame, rock_data
)
# (v,w), waypoint_local = planner.plan_arc(waypoint, current_pose, test_rock_points_rover_frame, test_rock_data)
# print(waypoint_local)
fig = plot_rocks_rover_frame(rock_points_rover_frame, rock_data, color="red")
print(f"waypoint_local{waypoint_local}")
# fig = go.Figure()
for arc in arcs:
    fig = plot_path_rover_frame(arc, fig=fig)
fig = plot_path_rover_frame(best_arc, color="green", fig=fig)

fig.show()
print(control)

In [ ]:
planner = ArcPlanner(11, 6.0)
arcs = planner.np_candidate_arcs
vw_poses_dict = {}
data_path = Path("../../../results/dynamics/v0.2_w0.0.json")
data_path = Path("../../../results/dynamics/v0.2_w0.0_scaled2.json")

vals = np.linspace(-0.4, 0.4, num=5)
vals = np.round(vals, 2)


for i in vals:
    path = f"../../../results/dynamics/v0.2_w{i:.1f}_scaled2.json"

    initial_pose, lander_pose, poses, imu_data, cam_config = load_data(path, dynamics=True)
    poses = np.array(poses)
    inverse_matrix = np.linalg.inv(initial_pose)

    rotated_poses = inverse_matrix @ poses
    vw_poses_dict[(0.2, i)] = rotated_poses[100:400, :2, 3]

    if i == -0.4:
        fig = plot_path_rover_frame(vw_poses_dict[(0.2, -0.4)])

    plot_path_rover_frame(vw_poses_dict[(0.2, i)], fig=fig)

for arc in arcs:
    fig = plot_path_rover_frame(arc, fig=fig, color="red")
fig.show()
# testing 0.5

In [ ]:
import pickle

with open(
    "../../../results/planner_stats/path_planner_stats_arc20_12.0s_scale2_rad0.75_replan20.pkl",
    "rb",
) as f:
    data = pickle.load(f)
print(len(data["collision detections"]))
print(data)

# Arc overlay visualization


In [ ]:
from lac.utils.camera import Camera
from lac.utils.frames import get_cam_pose_rover, CAMERA_TO_OPENCV_PASSIVE
from lac.utils.plotting import plot_poses

In [ ]:
def rover_pose_to_cam_pose(rover_pose, cam_name="FrontLeft"):
    camera_pose = get_cam_pose_rover(cam_name)
    camera_pose[:3, :3] = CAMERA_TO_OPENCV_PASSIVE
    return rover_pose @ camera_pose

In [ ]:
# data_path = Path("../../output/AutoAgent/arc_planner_test")
data_path = Path("../../output/NavAgent/map1_preset4_eval")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
cam_poses = []
for pose in poses:
    cam_poses.append(rover_pose_to_cam_pose(pose))

In [ ]:
plot_poses(cam_poses[::100])

In [ ]:
cam = Camera(cam_poses[0])

# TODO: define arc points
arc_points = None

uv, depths = cam.project_world_points_to_uv(arc_points)

In [ ]:
plt.scatter(uv[:, 0], uv[:, 1], cmap="plasma")
# Set x and y limits
plt.xlim(0, 1280)
plt.ylim(0, 720)
plt.gca().invert_yaxis()
plt.show()